In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd
import matplotlib.patches as patches
%matplotlib qt

In [2]:
path='../Sphere'

In [3]:
#Load preprocessed data and targets
data = pd.read_csv(path+'/data/train/00001/columns_1000ms.csv')
targets = pd.read_csv(path+'/data/train/00001/targets.csv')

In [130]:
# Make a dataframe with just the 2d video bounding box values
boundings = data[['video_hallway_bb_2d_br_y_mean', 
             'video_hallway_bb_2d_tl_y_mean',
             'video_hallway_bb_2d_br_x_mean',
             'video_hallway_bb_2d_tl_x_mean',
             'video_kitchen_bb_2d_br_y_mean', 
             'video_kitchen_bb_2d_tl_y_mean',
             'video_kitchen_bb_2d_br_x_mean',
             'video_kitchen_bb_2d_tl_x_mean',
             'video_living_room_bb_2d_br_y_mean', 
             'video_living_room_bb_2d_tl_y_mean',
             'video_living_room_bb_2d_br_x_mean',
             'video_living_room_bb_2d_tl_x_mean']]

accelerations = data[['acceleration_x_mean',
                     'acceleration_y_mean',
                     'acceleration_x_mean']]


In [128]:
# Create list of labels for each 1 second interval using argmax on the targets from the annotators
labels = []
for i in range(len(targets)):
    
    # Determine the index of the label which has the largest target value in the list of actions
    label_ind = np.argmax(targets.iloc[i][2:])
    
    if label_ind == -1:
        label = 'unknown'
        
    # If not unknown, assign to label the name of the column which has the largest target value
    else:
        label = targets.columns[label_ind + 2]
        
    labels.append(label)
    

column_names = targets.columns[2:]

In [207]:
def animate():
    
    #################################
    #### set up axes and patches ####
    #################################

    fig = plt.figure(figsize=(10,6))
    gs = fig.add_gridspec(3, 3, height_ratios=[5,5,1])
    
    
    camera = fig.add_subplot(gs[0, :2])
    camera.set_xlim([0, 320])
    camera.set_ylim([0, 240])
    camera.set_title('Camera: click to pause')
    camera_name = camera.text(0.5,0.1,'unknown', ha="center",va="center", color='white',
                     transform=camera.transAxes, fontsize="large")
    im = camera.imshow(np.ones((320,240)), extent=[0, 320, 0, 240])
    camera.invert_xaxis()
 
    
    l_im = plt.imread(path+"/data/supplementary/living_room.png")
    h_im = plt.imread(path+"/data/supplementary/hallway.png")
    k_im = plt.imread(path+"/data/supplementary/kitchen.png")

 
    floor = fig.add_subplot(gs[1, :2])
    floor.set_title('Floorplan')
    floor.set_xlim([0, 320])
    floor.set_ylim([0, 240])
    f_im = plt.imread(path+"/data/supplementary/floors.png")
    floor.imshow(f_im, extent=[0, 320, 0, 240])
    
    person = patches.Ellipse(xy=(-100,-100), width=20, height=20, fc='y', ec='k',zorder=100)
    floor.add_patch(person)

    pose = fig.add_subplot(gs[:, 2])
    pose.set_title('Pose')
    pose_text = pose.text(0.5,0.05,'', ha="center",va="center", color='black', fontsize="medium")
    
    pose.barh(column_names, np.zeros(20), color = 'salmon')
    
    time = fig.add_subplot(gs[2,:2])
    time.imshow(np.zeros((100,boundings.shape[0])))
    time.set_ylim([0, 100])
    time.set_xlim([0, boundings.shape[0]])
    time.set_title("Time: click to change time")  
    
    axes = (camera, floor, pose, time)
    
    for ax in axes:
        if ax == pose:
            ax.axes.xaxis.set_ticks([])
#             ax.axes.yaxis.set_ticks([])

        else:      
            ax.axes.xaxis.set_ticks([])
            ax.axes.yaxis.set_ticks([])
        
    line, = camera.plot([], [], lw=3)
    
    tracker = time.axvline(0, 0, 1)
    clock = time.text(0.01, 0.5,'',color='white',ha="left",va="center",
                     transform=time.transAxes, fontsize="medium")
    clock_tot = time.text(0.99, 0.5,f'{boundings.shape[0]}',color='white',ha="right",va="center",
                     transform=time.transAxes, fontsize="medium")
    
    paused = [ False ]
    last_index = [ -1 ]
    t_index = [ 0 ]
    
    ######################
    #### move patches ####
    ######################

    def draw(index):

        if not paused[0]:
            t_index[0] = t_index[0] + (index - last_index[0])
            t_index[0] = t_index[0] % boundings.shape[0]

        last_index[0] = index

        # set patches 
        bb_indexes = np.where(np.isnan(boundings.iloc[t_index[0]])==False)[0]
        if bb_indexes.size > 0:

            bbs = boundings.iloc[t_index[0]][bb_indexes]

            if bb_indexes[0] == 0:
                im.set_array(h_im)
                camera_name.set_text('Hallway')
                person.center = 160, 150

            elif bb_indexes[0] == 4:
                im.set_array(k_im)
                camera_name.set_text('Kitchen')
                person.center = 135, 55

            elif bb_indexes[0] == 8:
                im.set_array(l_im)
                camera_name.set_text('Living Room')
                person.center = 115, 180

            x_vals = [bbs[2],bbs[2],bbs[3],bbs[3],bbs[2]]
            #y_vals = 240-[bbs[0],bbs[1],bbs[1],bbs[0],bbs[0]]
            #x_vals = [320-bbs[3],320-bbs[3],320-bbs[2],320-bbs[2],320-bbs[3]]
            y_vals = [240-bbs[1],240-bbs[0],240-bbs[0],240-bbs[1],240-bbs[1]]
            line.set_data(x_vals, y_vals)
            line.set_color('yellow')
            
            heights = targets.iloc[t_index[0]][2:]
            
            pose.barh(column_names, heights, color = 'salmon')
        
            for bar in pose.containers:
                bar.remove()
            

            for j, b in enumerate(pose.containers):
                b[0].set_height(heights[j])
                
            pose.set_title(labels[t_index[0]])

#             pose.axes.tick_params(labelsize=12)
            
            
        else:
            line.set_data([],[])
            im.set_array(np.ones((320,240)))
            camera_name.set_text('unknown')
            person.center = -100,-100
        
#         pose_text.set_text(labels[t_index[0]])
        tracker.set_xdata([t_index[0], t_index[0]])
        clock.set_text(f'{t_index[0]}')
        
        super_tuple = (person, pose, clock, camera_name, im, tracker, line,) #line, # tuple of patches tuple(l_patches)+(trail, clock, tracker, intensity, location, angle, score) + agent_patches
        return super_tuple
    
    
    #################################
    #### set up animation params ####
    #################################
    

    def init():
        result = draw(0)
        return result

    
    def onclick(event):
        if event.button == 1:
            # pause if the user clicks on the main figure
            if event.inaxes is camera:
                paused[0] = not paused[0]
            # edit time directly if the user clicks on the graph over time
            elif event.inaxes is time:
                t_index[0] = (int) (event.xdata)            

                
    def anim(index):
        return draw(index)


    ani = FuncAnimation(fig, anim, init_func=init, frames=boundings.shape[0], interval=100, blit=False, repeat=False)

    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()    
    return ani

In [208]:
############################################################
############################################################
ani = animate()
############################################################
############################################################

In [117]:
HTML(ani.to_html5_video())

## Arrows - doesn't work

In [209]:
def animate():
    
    #################################
    #### set up axes and patches ####
    #################################

    fig = plt.figure(figsize=(10,6))
    gs = fig.add_gridspec(3, 3, height_ratios=[5,5,1])
    
    
    camera = fig.add_subplot(gs[0, :1])
    camera.set_xlim([0, 320])
    camera.set_ylim([0, 240])
    camera.set_title('Camera: click to pause')
    camera_name = camera.text(0.5,0.1,'unknown', ha="center",va="center", color='white',
                     transform=camera.transAxes, fontsize="large")
    im = camera.imshow(np.ones((320,240)), extent=[0, 320, 0, 240])
    camera.invert_xaxis()
 
    
    l_im = plt.imread(path+"/data/supplementary/living_room.png")
    h_im = plt.imread(path+"/data/supplementary/hallway.png")
    k_im = plt.imread(path+"/data/supplementary/kitchen.png")

 
    floor = fig.add_subplot(gs[1, :1])
    floor.set_title('Floorplan')
    floor.set_xlim([0, 320])
    floor.set_ylim([0, 240])
    f_im = plt.imread(path+"/data/supplementary/floors.png")
    floor.imshow(f_im, extent=[0, 320, 0, 240])
    
    person = patches.Ellipse(xy=(-100,-100), width=20, height=20, fc='y', ec='k',zorder=100)
    floor.add_patch(person)
    
    acc = fig.add_subplot(gs[:2,1])
    acc.set_title('Acceleration')
    acc.set_xlim([-1.1,1.1])
    acc.set_ylim([-1.1,1.1])
#     acc.arrow(0,0,-1,0, width = 0.02, head_width=0.05,color = 'black')
    arrow = plt.arrow(0,0,-1,0, width = 0.02, head_width=0.05,color = 'black')
    acc.add_patch(arrow)

    pose = fig.add_subplot(gs[:, 2])
    pose.set_title('Pose')
    pose_text = pose.text(0.5,0.05,'', ha="center",va="center", color='black', fontsize="medium")
    
    pose.barh(column_names, np.zeros(20), color = 'salmon')
    
    time = fig.add_subplot(gs[2,:2])
    time.imshow(np.zeros((100,boundings.shape[0])))
    time.set_ylim([0, 100])
    time.set_xlim([0, boundings.shape[0]])
    time.set_title("Time: click to change time")  
    
    axes = (camera, floor, pose, time, acc)
    
    for ax in axes:
        if ax == pose:
            ax.axes.xaxis.set_ticks([])
#             ax.axes.yaxis.set_ticks([])

        else:      
            ax.axes.xaxis.set_ticks([])
            ax.axes.yaxis.set_ticks([])
        
    line, = camera.plot([], [], lw=3)
    
    tracker = time.axvline(0, 0, 1)
    clock = time.text(0.01, 0.5,'',color='white',ha="left",va="center",
                     transform=time.transAxes, fontsize="medium")
    clock_tot = time.text(0.99, 0.5,f'{boundings.shape[0]}',color='white',ha="right",va="center",
                     transform=time.transAxes, fontsize="medium")
    
    paused = [ False ]
    last_index = [ -1 ]
    t_index = [ 0 ]
    
    ######################
    #### move patches ####
    ######################

    def draw(index):

        if not paused[0]:
            t_index[0] = t_index[0] + (index - last_index[0])
            t_index[0] = t_index[0] % boundings.shape[0]

        last_index[0] = index

        # set patches 
        bb_indexes = np.where(np.isnan(boundings.iloc[t_index[0]])==False)[0]
        if bb_indexes.size > 0:

            bbs = boundings.iloc[t_index[0]][bb_indexes]

            if bb_indexes[0] == 0:
                im.set_array(h_im)
                camera_name.set_text('Hallway')
                person.center = 160, 150

            elif bb_indexes[0] == 4:
                im.set_array(k_im)
                camera_name.set_text('Kitchen')
                person.center = 135, 55

            elif bb_indexes[0] == 8:
                im.set_array(l_im)
                camera_name.set_text('Living Room')
                person.center = 115, 180

            x_vals = [bbs[2],bbs[2],bbs[3],bbs[3],bbs[2]]
            #y_vals = 240-[bbs[0],bbs[1],bbs[1],bbs[0],bbs[0]]
            #x_vals = [320-bbs[3],320-bbs[3],320-bbs[2],320-bbs[2],320-bbs[3]]
            y_vals = [240-bbs[1],240-bbs[0],240-bbs[0],240-bbs[1],240-bbs[1]]
            line.set_data(x_vals, y_vals)
            line.set_color('yellow')
            
            heights = targets.iloc[t_index[0]][2:]
            
            pose.barh(column_names, heights, color = 'salmon')
        
            for bar in pose.containers:
                bar.remove()
            

            for j, b in enumerate(pose.containers):
                b[0].set_height(heights[j])
                
            pose.set_title(labels[t_index[0]])
            
#             acc.remove()
            
#             acc.arrow(0,0,accs[t_index[0]][0],accs[t_index[0]][1])
            
#             acc.arrow(0,0,0,accs[t_index[0]][1])
#             acc.add_patch(arrow)
#             acc.add_patch(yarrow)
            
            
        else:
            line.set_data([],[])
            im.set_array(np.ones((320,240)))
            camera_name.set_text('unknown')
            person.center = -100,-100
        
#         pose_text.set_text(labels[t_index[0]])
        tracker.set_xdata([t_index[0], t_index[0]])
        clock.set_text(f'{t_index[0]}')
        
        super_tuple = (person, pose, acc, clock, camera_name, im, tracker, line,) #line, # tuple of patches tuple(l_patches)+(trail, clock, tracker, intensity, location, angle, score) + agent_patches
        return super_tuple
    
    
    #################################
    #### set up animation params ####
    #################################
    

    def init():
        result = draw(0)
        return result

    
    def onclick(event):
        if event.button == 1:
            # pause if the user clicks on the main figure
            if event.inaxes is camera:
                paused[0] = not paused[0]
            # edit time directly if the user clicks on the graph over time
            elif event.inaxes is time:
                t_index[0] = (int) (event.xdata)            

                
    def anim(index):
        return draw(index)


    ani = FuncAnimation(fig, anim, init_func=init, frames=boundings.shape[0], interval=100, blit=False, repeat=False)

    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()    
    return ani

In [210]:
############################################################
############################################################
ani = animate()
############################################################
############################################################

In [125]:
# def animate(self, poses, sensations, states, scores):
#     r, l_pos = self.agent_radius, [0,0,0.1] #self.light_start
#     x, y, theta = poses[0]
#     x1, y1, z1 = l_pos
#     thetat = 0

#     # use an Ellipse to visually represent the agent's body
#     body = patches.Ellipse(xy=(0, 0), width=2 * r, height=2 * r, fc='w', ec='k',zorder=100)
#     # use a black dot to visually represent each sensor
#     sensors = [ patches.Ellipse(xy=(r * np.cos(theta), r * np.sin(theta)), width=0.5, height=0.5, fc='b', zorder=50) for theta in self.sensors ]
#     # use small rectangles to visually represent the motors
#     motors = [ patches.Rectangle((-0.5*r, y), width = r, height = 0.2*r, color="black", zorder=130) for y in (-1.1*r, 0.9*r) ]
#     # use a line to indicate the agent's orientation
#     line = Line2D( (x, x + r * np.cos(theta)), (y, y + r * np.sin(theta)), linewidth=2, color='black')
#     #line = Line2D( (0, r), (0, 0) )

#     # draw a line showing the agent's "trail"
#     trail = Line2D( [], [], color='r') 
#     # display a clock
#     clock = Annotation('', (0.75, 0.9), xycoords='axes fraction')
#     intensity = Annotation('', (0.75, 0.8), xycoords='axes fraction')  
#     location = Annotation('', (0.15, 0.9), xycoords='axes fraction')  
#     angle = Annotation('', (0.75, 0.7), xycoords='axes fraction')
#     score = Annotation('', (0.75, 0.9), xycoords='axes fraction')


#     fig = plt.figure(figsize=(6,8))
#     gs1 = gridspec.GridSpec(3, 1, height_ratios=[8,1,3])

#     ax1 = fig.add_subplot(gs1[0])
#     ax1.axis("equal")


#     ########################
#     ax1.set_xlim([-self.zoom, self.zoom])
#     ax1.set_ylim([-self.zoom, self.zoom])
#     ########################

#     ax1.set_title("Click On Main Display To Pause / Unpause")

#     ax2 = fig.add_subplot(gs1[1])
#     ax2.set_title("Click On Sensor Graph To Change Time")

#     tracker = ax2.axvline(0, 0, 1)
#     paused = [ False ]
#     last_index = [ -1 ]
#     t_index = [ 0 ]

#     if sensations is not None:
#         times = np.arange(0, self.dt * len(sensations), self.dt)
#         # plot the sensor values
#         ax2.plot(times, sensations, 'k');
#         # plot the ideal (noiseless) sensor values
#         #ideal = np.array([self.sensor_transform(self.sensor_input(pose)) for pose in poses[:-1]])
#         #ax2.plot(times, ideal, 'r')


#     # A: Parameters of Likelihood of generative model
#     # B: Parameters of Prior of generative model
#     # C: Parameters of Prior beliefs about future observations

#     gs2 = gs1[2].subgridspec(1, 4, width_ratios=[2,2,2,1])


#     gs1.tight_layout(fig, pad=0, h_pad=2, w_pad=2, rect=(0,0,1,0.85))

#     def draw(index):
#         if not paused[0]:
#             t_index[0] = t_index[0] + (index - last_index[0])
#             t_index[0] = t_index[0] % len(poses)

#         last_index[0] = index

#         x, y, theta = poses[t_index[0]]

# #             B_index = states[t_index[0]]['B_index']
# #             B = np.zeros((2,2,2))
# #             B[B_index]=1

# #             img2.set_data(B[0])
# #             img3.set_data(B[1])

# #             stat = states[t_index[0]]['mdp_matrices']
# #             B0=np.ravel(stat[1][0])
# #             B1=np.ravel(stat[1][1])

# #             for t in range(len(im2.texts)):
# #                 im2.texts[t].set_text('{:.3f}'.format(B0[t]))
# #                 im3.texts[t].set_text('{:.3f}'.format(B1[t]))

#         thetat = ((theta*360/(2*np.pi))%360)

#         tr = Affine2D().rotate(theta).translate(x, y) + ax1.transData

#         agent_patches = (body, line) + tuple(sensors) + tuple(motors)

#         for patch in agent_patches:
#             patch.set_transform(tr);


#         l_patches = []
#         for l in self.lights:
#             l_patches.append(l.patch_y)
#             l_patches.append(l.patch_y2)
#             l_patches.append(l.patch_y3)
#             l_patches.append(l.patch_b)

#             l.move_light(t_index[0], ax1)
#             l.change_luminance(t_index[0], ax1)


#         trail.set_data( poses[:t_index[0], 0], poses[:t_index[0], 1] )

#         time = t_index[0] * self.dt
#         tracker.set_xdata([time, time])


#         #clock.set_text("Time: %.02f" % time)
#         score.set_text("Score: {:.2f}".format(scores[t_index[0]]))
#         intensity.set_text("Intensity: %.02f" % sum(sensations[t_index[0]]))
#         #location.set_text("Location: {:.2f} {:.2f}".format(x, y))
#         #angle.set_text("Angle: %.02f" % thetat)

#         super_tuple = tuple(l_patches)+(trail, clock, tracker, intensity, location, angle, score) + agent_patches
#         return super_tuple

#     def init():
#         result = draw(0)
#         for artist in result:
#             if artist is not tracker:
#                 ax1.add_artist(artist)
#         return result

#     def onclick(event):
#         if event.button == 1:
#             # pause if the user clicks on the main figure
#             if event.inaxes is ax1:
#                 paused[0] = not paused[0]
#             # edit time directly if the user clicks on the graph over time
#             elif event.inaxes is ax2:
#                 t_index[0] = (int) (event.xdata / self.dt)            

#     def anim(index):
#         return draw(index)


#     ani = FuncAnimation(fig, anim, init_func=init, frames = None, interval=self.interval*self.dt/self.speedup, blit=True, save_count=len(poses))

#     fig.canvas.mpl_connect('button_press_event', onclick)
#     #plt.tight_layout()
#     plt.show()    
#     return ani